In [2]:
import time
import pandas as pd
import os
import glob

from pytrends.request import TrendReq
from datetime import datetime, timedelta

In [10]:
# read data
input_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/searching_index/results'

os.chdir(input_path)
file_ls = glob.glob('results_*')
data = pd.concat(pd.read_csv(file) for file in file_ls)

In [11]:
data.sample(5)

,Unnamed: 0.1,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,...,domestic_gross,worldwide_gross,distributor,mpaa_rating,genre,movie_same_name,averageRating,numVotes,search_index,error_message
22,1222.0,24589,tt3960412,movie,Popstar: Never Stop Never Stopping,Popstar: Never Stop Never Stopping,0,2016,\N,87,...,9496130.0,9537120.0,Universal,R,Comedy,1,6.7,66695.0,NaN,"HTTPSConnectionPool(host='trends.google.com', ..."
42,242.0,1865,tt0310778,movie,Bon Voyage,Bon voyage,0,2003,\N,114,...,2353728.0,8361736.0,Sony Pictures,PG-13,Comedy,1,6.7,5008.0,68.545455,NaN
182,982.0,11478,tt1392190,movie,Mad Max: Fury Road,Mad Max: Fury Road,0,2015,\N,120,...,153636354.0,370651733.0,Warner Bros.,R,Action,1,8.1,1055607.0,4.653846,NaN
57,1157.0,3151,tt0479968,movie,One Missed Call,One Missed Call,0,2008,\N,87,...,26890041.0,44513466.0,Warner Bros.,PG-13,Horror,1,4.0,33767.0,5.096154,NaN
85,685.0,3160,tt0480687,movie,Hall Pass,Hall Pass,0,2011,\N,105,...,45060734.0,87173475.0,Warner Bros.,R,Comedy,1,5.9,129669.0,6.634615,NaN


In [15]:
# filter data, and extract the searching index again
filtered_df = data[data['error_message'].str.contains("429|443", na=False)]
original_df = data[~(data['error_message'].str.contains("429|443", na=False))]

results_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/searching_index/results_supplement'
os.chdir(results_path)

original_df.to_csv('results_w_searching_index.csv')

In [16]:
pytrends = TrendReq(hl='en-US', tz=360)

In [17]:
def fetch_trend(movie_title, release_date):
    # calculate the start date
    start_date = release_date - timedelta(days=365)

    # change the datetime format
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = release_date.strftime('%Y-%m-%d')

    # build the payload
    pytrends.build_payload(kw_list=[movie_title], timeframe=f'{start_date_str} {end_date_str}')

    # fetch the interest over time
    trends = pytrends.interest_over_time()

    # return the mean (we can change mean to other statistics)
    return trends[movie_title].mean()

In [18]:
data_sample = filtered_df.drop(columns=['search_index', 'error_message'])

for index, row in data_sample.iterrows():
    try: 
        # convert to datetime object
        release_date = pd.to_datetime(row['release_date'])

        # retrieve the data
        trend_index = fetch_trend(row['primaryTitle'], release_date)

        # store the data
        data_sample.at[index, 'search_index'] = trend_index

        # to avoid the error that reach the maximum retrieve matrix
        time.sleep(15)
    except Exception as e:
        print(f'error: {e}')
        data_sample.at[index, 'error_message'] = e
        time.sleep(15)
        continue

error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 429
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 429
error: The request failed: Google returned a response with code 429
error: 'Joe'
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 429
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google returned a response with code 500
error: The request failed: Google r

In [19]:
os.chdir(results_path)
data_sample.to_csv('results_w_supplement_searching_index.csv')